# Building Machine Learning Classifiers: Evaluate Random Forest with GridSearchCV

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Read in text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Exploring parameter settings using GridSearchCV

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV 

In [7]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
         'max_depth': [30, 60, 90, None]}
gs = GridSearchCV(rf, param, cv= 5, n_jobs=-1)         
gs_fit = gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending = False)[0:5]

C:\Users\Mahamat Nour\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Mahamat Nour\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Mahamat Nour\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Mahamat Nour\Anaconda3\li

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
8,60.491088,4.893583,0.633037,0.113110,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.978475,0.978437,0.973046,...,0.974313,0.003761,1,0.999326,0.999102,0.999102,0.999775,0.999102,0.999281,0.000262
7,23.746573,0.678962,0.340004,0.012082,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.977578,0.977538,0.973046,...,0.973415,0.003674,2,0.999326,0.998877,0.999326,0.999775,0.999102,0.999281,0.000298
10,30.172321,4.190376,0.567888,0.135511,None,150,"{'max_depth': None, 'n_estimators': 150}",0.978475,0.976640,0.973944,...,0.973056,0.004375,3,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
5,35.294547,1.715407,0.482523,0.109854,60,300,"{'max_depth': 60, 'n_estimators': 300}",0.975785,0.975741,0.973046,...,0.972696,0.003048,4,0.994160,0.993489,0.994612,0.993264,0.993938,0.993893,0.000479
11,65.276445,7.075949,0.807666,0.140858,None,300,"{'max_depth': None, 'n_estimators': 300}",0.975785,0.975741,0.973046,...,0.972696,0.003048,4,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000


In [9]:
rf_count = RandomForestClassifier()

param = {'n_estimators': [10, 150, 300],
         'max_depth': [30, 60, 90, None]}
gs_count= GridSearchCV(rf_count, param, cv= 5, n_jobs=-1)
gs_fit_count = gs.fit(X_count_feat, data['label'])
pd.DataFrame(gs_fit_count.cv_results_).sort_values('mean_test_score', ascending = False)[0:5]

C:\Users\Mahamat Nour\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Mahamat Nour\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Mahamat Nour\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Mahamat Nour\Anaconda3\li

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
7,21.379731,0.598110,0.344003,0.041996,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.979372,0.975741,0.973046,...,0.973594,0.003721,1,0.999102,0.998653,0.999551,0.998877,0.998877,0.999012,0.000305
10,22.904656,0.285693,0.363791,0.022905,None,150,"{'max_depth': None, 'n_estimators': 150}",0.978475,0.978437,0.974843,...,0.973235,0.005103,2,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
6,2.871951,0.076587,0.176898,0.007453,90,10,"{'max_depth': 90, 'n_estimators': 10}",0.973991,0.978437,0.972147,...,0.972517,0.003798,3,0.996406,0.997979,0.997755,0.998204,0.996408,0.997350,0.000783
8,41.051244,0.783922,0.492117,0.076315,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.977578,0.973046,0.973944,...,0.972337,0.003441,4,0.998652,0.998877,0.998877,0.999102,0.998877,0.998877,0.000142
3,2.728034,0.176582,0.182896,0.011535,60,10,"{'max_depth': 60, 'n_estimators': 10}",0.972197,0.973046,0.972147,...,0.971978,0.002920,5,0.991914,0.990795,0.992366,0.995959,0.992142,0.992635,0.001747


In [10]:
tokenized = ['test','in','the','rest','of','for','new','last']
result = [word for word in tokenized if word not in ['in','on','the','of','for']]
print(result)

['test', 'rest', 'new', 'last']
